In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np
import datetime as dt
import seaborn as sns
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_csv('/kaggle/input/ecommerce-data/data.csv', header= 0, encoding= 'unicode_escape')
data.head()

In [ ]:
data.isnull().sum()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include='O')

In [ ]:
data= data[pd.notnull(data['CustomerID'])]

In [ ]:
data.isnull().sum()

In [ ]:
df=data[['Country','CustomerID']].drop_duplicates()

In [ ]:
df.shape

In [ ]:
data.head()

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.Country.value_counts()[:10].plot(kind='bar')

In [ ]:
uk = data[data.Country=='United Kingdom']

In [ ]:
uk.info()

In [ ]:
uk.describe()

In [ ]:
uk = uk[(uk['Quantity']>0)]
uk.info()

In [ ]:
uk=uk[['CustomerID','InvoiceDate','InvoiceNo','Quantity','UnitPrice']]

In [ ]:
uk.head()

In [ ]:
uk['TotalPrice'] = uk['Quantity'] * uk['UnitPrice']
uk.head()

In [ ]:
uk['InvoiceDate'].min(),uk['InvoiceDate'].max()

In [ ]:
now = dt.datetime(2011,12,10)
uk['InvoiceDate'] = pd.to_datetime(uk['InvoiceDate'])
uk.head()

In [ ]:
calc = uk.groupby('CustomerID').agg({'InvoiceDate':lambda x:x.min().month,
                                     'InvoiceNo':lambda x:len(x),
                                     'TotalPrice':lambda x:np.sum(x)})
calc.columns  = ['Start_Month', 'Frequency', 'Total_Sales']
calc.head()

In [ ]:
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
profit_margin = 0.07 #assuming this. Could use the genuine value if provided by the organization.
mon_CLV = []
for i in range(1,13):
    cust = calc[calc['Start_Month'] == i]
    avg_sales = round(np.mean(cust['Total_Sales']),2)
    pur_freq = round(np.mean(cust['Frequency']),2)
    ret_rate = cust[cust['Frequency']>1].shape[0]/cust.shape[0]
    churn = round(1-ret_rate,2)
    CLV = round(((avg_sales*pur_freq/churn))*profit_margin,2)
    mon_CLV.append(CLV) 
pd.DataFrame([months,mon_CLV])

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

In [ ]:
X = uk[['UnitPrice','TotalPrice','CustomerID']]
y = uk['Quantity']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10)

In [ ]:
def get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test):
    model = DecisionTreeRegressor(max_leaf_nodes=max_leaf_nodes, random_state=0)
    model.fit(X_train, y_train)
    preds_val = model.predict(X_test)
    mae = mean_absolute_error(y_test, preds_val)
    return(mae)

candidate_max_leaf_nodes = [5, 25, 50, 100, 250, 500]

for max_leaf_nodes in candidate_max_leaf_nodes:
    my_mae = get_mae(max_leaf_nodes, X_train, X_test, y_train, y_test)
    print("Max leaf nodes: %d  \t\t Mean Absolute Error:  %d" %(max_leaf_nodes, my_mae))

In [ ]:
dtr = DecisionTreeRegressor(max_leaf_nodes=500) # <- default number of lead nodes is unlimited

dtr.fit(X_train, y_train)
pred_dtr = dtr.predict(X_test)
score_dtr_per = dtr.score(X_test, y_test) * 100
print(f'Decision Tree Regressor Model Score: {score_dtr_per}%')